In [1]:
import bw2data as bd
import bw2regional as bwr
import bw2_lcimpact as lcia
import bw2_aware as aware
from bw2regional.pandarus_remote import AlreadyExists
import rower
import os

In [2]:
assert lcia.__version__ >= (0, 4, 2)

In [2]:
bd.projects.set_current("LC IMPACT case study")

# Add basic metadata

Such as country definitions

In [5]:
bwr.create_world_collections()

In [6]:
bwr.create_ecoinvent_collections()

In [7]:
bwr.create_restofworlds_collections()

Creating `rower` 'RoW' geo/topocollections


# Fix ecoinvent Rest-of-World definitions

In [8]:
rower_obj = rower.Rower('ecoinvent')

In [9]:
rower_obj.load_existing(rower_obj.EI_3_8_CUTOFF)
rower_obj.label_RoWs()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:40


3977

# Fix location labels to include geocollections

In [10]:
bwr.label_activity_geocollections("Fuel comparison")

0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [11]:
bwr.label_activity_geocollections("ecoinvent")

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:37


In [13]:
bd.databases['biosphere3']['geocollections'] = []
bd.databases.flush()

In [14]:
bwr.geocollections

Geocollections dictionary with 3 object(s):
	RoW
	ecoinvent
	world

In [15]:
for name in bd.databases:
    print(name, bd.databases[name]['geocollections'])

biosphere3 []
ecoinvent ['RoW', 'ecoinvent', 'world']
Fuel comparison ['ecoinvent']


# Add regionalized LC IMPACT and AWARE impact categories

In [16]:
lcia.import_regionalized_lcimpact()
lcia.create_regionalized_intersections()

Merging topographical faces for geocollection ecoinvent
Creating intersection (ecoinvent, watersheds-hh)
Merging topographical faces for geocollection RoW
Creating intersection (RoW, watersheds-hh)
Merging topographical faces for geocollection world
Creating intersection (world, watersheds-hh)
Merging topographical faces for geocollection ecoinvent
Creating intersection (ecoinvent, watersheds-eq-sw-certain)
Merging topographical faces for geocollection RoW
Creating intersection (RoW, watersheds-eq-sw-certain)
Merging topographical faces for geocollection world
Creating intersection (world, watersheds-eq-sw-certain)
Merging topographical faces for geocollection ecoinvent
Creating intersection (ecoinvent, watersheds-eq-sw-all)
Merging topographical faces for geocollection RoW
Creating intersection (RoW, watersheds-eq-sw-all)
Merging topographical faces for geocollection world
Creating intersection (world, watersheds-eq-sw-all)
Merging topographical faces for geocollection ecoinvent
Creat

In [3]:
aware.import_regionalized_aware()
aware.create_regionalized_intersections()

Calculation job submitted.

Job ended with status 'finished'
Merging topographical faces for geocollection ecoinvent
Creating intersection (ecoinvent, world-topo-watersheds-aware)
Merging topographical faces for geocollection world
Creating intersection (world, world-topo-watersheds-aware)
Merging topographical faces for geocollection RoW
Creating intersection (RoW, world-topo-watersheds-aware)
Skipping existing intersection: (world-topo-watersheds-aware, watersheds-aware)


# Calculate intersections between inventory and IA spatial scales

In [19]:
CATEGORIES = [
    "watersheds-hh",
    "watersheds-eq-sw-certain",
    "watersheds-eq-sw-all",
    "particulate-matter",
    "ecoregions",
]

In [17]:
for category in CATEGORIES:
    bwr.calculate_intersection("world", category)

In [ ]:
for category in CATEGORIES:
    remote.intersection("world", category)

## Get third spatial scale

This is the intersection of inventory and IA spatial scales

In [ ]:
for category in CATEGORIES:
    remote.intersection_as_new_geocollection('world', category, 'world-topo-' + category)

In [ ]:
CATEGORIES = [
    "watersheds-hh",
    "watersheds-eq-sw-certain",
    "watersheds-eq-sw-all",
    "particulate-matter",
    "ecoregions",
]

for category in CATEGORIES:
    try:
        remote.calculate_intersection("world-topo-" + category, category)
    except AlreadyExists:
        pass

In [ ]:
for category in CATEGORIES:
    remote.intersection("world-topo-" + category, category)

# Add extension table data

## Add raster `geocollections`

In [21]:
rasters = [
    'maize_landuse_intensity',
    'maize_water_intensity',
    'sugarcane_landuse_intensity',
    'sugarcane_water_intensity',
    'gdpweighted',
]

for label in rasters:
    fp = os.path.abspath(os.path.join("..", "data", label + ".tif"))
    assert os.path.isfile(fp)

    if label == "gdpweighted":
        label = 'weighted-pop-density'
    
    bwr.geocollections[label] = {
        'filepath': fp,
        'band': 1
    }

## Define raster statistics

Need statistics for the spatial scale which is the intersection of inventory and IA

In [23]:
inters = [
    'world-topo-watersheds-hh',
    'world-topo-watersheds-eq-sw-certain',
    'world-topo-watersheds-eq-sw-all',
    'world-topo-particulate-matter',
    'world-topo-ecoregions',
]

crop_rasters = [
    'maize_landuse_intensity',
    'maize_water_intensity',
    'sugarcane_landuse_intensity',
    'sugarcane_water_intensity',
    'weighted-pop-density',
]

for x in inters:
    for y in crop_rasters:
        try:
            bwr.remote.calculate_rasterstats(x, y)
        except AlreadyExists:
            pass

## Import raster statistics as extension tables

In [25]:
for x in inters:
    for y in crop_rasters:
        bwr.remote.rasterstats_as_xt(x, y, x + "-" + y)

In [26]:
bwr.extension_tables

ExtensionTables dictionary with 25 objects, including:
	world-topo-ecoregions-maize_landuse_intensity
	world-topo-ecoregions-maize_water_intensity
	world-topo-ecoregions-sugarcane_landuse_intensity
	world-topo-ecoregions-sugarcane_water_intensity
	world-topo-ecoregions-weighted-pop-density
	world-topo-particulate-matter-maize_landuse_intensity
	world-topo-particulate-matter-maize_water_intensity
	world-topo-particulate-matter-sugarcane_landuse_intensity
	world-topo-particulate-matter-sugarcane_water_intensity
	world-topo-particulate-matter-weighted-pop-density
Use `list(this object)` to get the complete list.